In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_excel("Coffee Shop Sales.xlsx")

In [8]:
missingVals = data.isnull().sum()
print(missingVals)

transaction_id      0
transaction_date    0
transaction_time    0
transaction_qty     0
store_id            0
store_location      0
product_id          0
unit_price          0
product_category    0
product_type        0
product_detail      0
dtype: int64


In [9]:
duplicates = data.duplicated().sum()
cleanedData = data.drop_duplicates()

In [11]:
for c in cleanedData.select_dtypes(include=['object']).columns:
 cleanedData[c] = cleanedData[c].astype(str)
for c in cleanedData.select_dtypes(include=['object']).columns:
 cleanedData[c] = cleanedData[c].str.strip()
cleanedData['transaction_time'] = pd.to_datetime(cleanedData['transaction_time'], format='%H:%M:%S')

In [13]:
cleanedData.to_excel("Cleaned_Coffee_Shop_Sales.xlsx", index=False)

In [18]:
cleanedData['total_sales'] = cleanedData['transaction_qty'] * cleanedData['unit_price']
prodSummary = cleanedData.groupby('product_detail').agg({
 'transaction_qty': 'sum',
 'unit_price': 'mean',
 'total_sales': 'sum'
}).reset_index()
prodSummary['profit'] = prodSummary['total_sales'] - (prodSummary['transaction_qty'] * prodSummary['unit_price'])

In [20]:
productsMakingProfits = prodSummary[prodSummary['profit'] > 0]
productsCausingLoss = prodSummary[prodSummary['profit'] <= 0]

In [22]:
topProducts = productsMakingProfits.sort_values(by='profit', ascending=False)
top10 = topProducts.sort_values(by='profit', ascending=False)
bottomProducts = productsCausingLoss.sort_values(by='profit', ascending=True)
bottom10 = productsCausingLoss.sort_values(by='profit', ascending=True)

In [24]:
feat = prodSummary[['transaction_qty', 'unit_price', 'total_sales']]
target = prodSummary['profit']
X_train, X_test, y_train, y_test = train_test_split(feat, target, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [26]:
profitPrediction = model.predict(X_test)
coef = pd.DataFrame({
 'Feat.': feat.columns,
 'Coef.': model.coef_
})

In [28]:
cleanedData['transaction_date'] = pd.to_datetime(cleanedData['transaction_date'])
dailySales = cleanedData.set_index('transaction_date').resample('D')['total_sales']
dailySales = dailySales.fillna(0)

In [30]:
prodSummary['profit_margin'] = (prodSummary['profit'] / prodSummary['total_sales'])
topMarginProds = prodSummary.sort_values(by='profit_margin', ascending=False)

In [34]:
trends = cleanedData.groupby(['product_detail', pd.Grouper(key='transaction_date', freq='M')]).agg({
    'total_sales': 'sum'
}).reset_index()
trends['monthly_growth'] = trends.groupby('product_detail')['total_sales'].pct_change()
growingProds = trends.groupby('product_detail').agg({
    'monthly_growth': 'mean'
}).reset_index()
growingProds = growingProds[growingProds['monthly_growth'] > 0]

In [39]:
topProducts = productsMakingProfits.sort_values(by='profit', ascending=False)

In [47]:
grossMarginPercentage = prodSummary['profit_margin'].mean()
lossFactors['unit_cost'] = lossFactors['unit_price'] * (1 - grossMarginPercentage)
lossFactors['total_cost'] = lossFactors['transaction_qty'] * lossFactors['unit_cost']
lossFactors['profit'] = lossFactors['total_sales'] - lossFactors['total_cost']
productsCausingLoss = lossFactors[lossFactors['profit'] < 0]